# Import des constantes

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

#Tokenization 
import nltk
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import TweetTokenizer
from functools import reduce
from operator import add
import collections

#Word2Vec
from gensim.models import Word2Vec
from multiprocessing import cpu_count

#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Import et nettoyage de la base

In [2]:
f = open('1377884570_tweet_global_warming.txt', 'r',newline='', encoding='ISO-8859-1')
content = f.read().split('\r')

content_new=[]
for x in content : 
    if len(x)>0:
        content_new.append(x)

content_new=content_new[1:len(content_new)]

In [3]:
col_tweet=[]
col_existence=[]
col_score=[]

#Split tweet , Note

for line in content_new:
    if len(line.split('[link]'))==2:
        (x,y)=line.split('[link]')
        col_tweet.append(x)
        col_existence.append(y)
    else : 
        if len(line.split(',Yes,'))==2:
            col_tweet.append(line.split(',Yes,')[0])
            col_existence.append(',Yes,'+line.split(',Yes,')[1])
        elif len(line.split(',No,'))==2:
            col_tweet.append(line.split(',No,')[0])
            col_existence.append(',No,'+line.split(',No,')[1])
        elif len(line.split(',Y,'))==2:
            col_tweet.append(line.split(',Y,')[0])
            col_existence.append(',Yes,'+line.split(',Y,')[1])
        elif len(line.split(',N/A,'))==2:
            col_tweet.append(line.split(',N/A,')[0])
            col_existence.append(',N/A,'+line.split(',N/A,')[1])
        elif len(line.split(',NA,'))==2:
            col_tweet.append(line.split(',NA,')[0])
            col_existence.append(',NA,'+line.split(',NA,')[1])
        elif len(line.split(',N,'))==2:
            col_tweet.append(line.split(',N,')[0])
            col_existence.append(',No,'+line.split(',N,')[1])
        else : 
            print('erreur')
            #print(line.split('[link]'))
col_tweet.append('I truly  Fat ASS Gore should get the Scam Artist Award of the decade with his Global Warming and Energy Credits worth close to Billion')
col_existence.append(' ,NA')
col_tweet.append('Despite Climategate, LEFT investing heavily in global warming hysteria as new way 2 impose nat\'l & international controls on human freedom.')
col_existence.append(' ,NA')
        
# Split Existence/Note
col_existence_new=[]

for x in col_existence:
    if len(x.split(','))==3:
        col_existence_new.append(x.split(',')[1])
        col_score.append(x.split(',')[2])
    else:
        col_existence_new.append('NA')
        col_score.append('NA')
        
#Nettoyage existence
for avis in range(len(col_existence_new)):
    if col_existence_new[avis]=='NA' or col_existence_new[avis]=='N/A' or col_existence_new[avis]=='':
        col_existence_new[avis]=np.nan
        
#Nettoyage score
for score in range(len(col_score)):
    if 'NA' not in col_score[score]:
        col_score[score]=col_score[score].split('\t')[0]
        if len(col_score[score].split('"'))>1:
            col_score[score]=float(col_score[score].split('"')[0])
        else: 
            col_score[score]=float(col_score[score])
            
    else : 
        col_score[score]=np.nan

#Creation du DataFrame
dic={'Tweet':col_tweet,'Existence':col_existence_new,'Score':col_score}
df=pd.DataFrame(dic)
    

erreur
erreur


In [4]:
df

,Tweet,Existence,Score
0,Global warming report urges governments to act...,Yes,1.0000
1,Fighting poverty and global warming in Africa,Yes,1.0000
2,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
3,Carbon offsets: How a Vatican forest failed to...,Yes,1.0000
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6085,Global warming you tube parody you will enjoy ...,No,0.6411
6086,One-Eyed Golfer: Don't dare tell me about glob...,No,1.0000
6087,man made global warming a hair brained theory ...,No,1.0000
6088,I truly Fat ASS Gore should get the Scam Arti...,NaN,NaN


# Quelques statistiques

In [5]:
print('On dispose de {} tweets.'.format(df.shape[0]))

print(' ')
print('On a {} données manquantes sur le label de l\'avis du tweet .'.format(str(df.isnull().sum()['Existence'])))
print(' ')

#Personnes convaincues du changement climatique 
print('On a {} tweets qui croit au Changement climatique.'.format(str(df[df.Existence=='Yes'].shape[0])))
m=round(df[df.Existence=='Yes'].Score.describe()['mean'],2)
std=round(df[df.Existence=='Yes'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))
print(' ')

#Personnes qui doutent du changement climatique 
print('On a {} tweets qui remettent en doute le Changement climatique.'.format(str(df[df.Existence=='No'].shape[0])))
m=round(df[df.Existence=='No'].Score.describe()['mean'],2)
std=round(df[df.Existence=='No'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))


On dispose de 6090 tweets.
 
On a 1865 données manquantes sur le label de l'avis du tweet .
 
On a 3111 tweets qui croit au Changement climatique.
Parmis ces personnes, le score est de 0.82 en moyenne, avec un écart-type de 0.18.
 
On a 1114 tweets qui remettent en doute le Changement climatique.
Parmis ces personnes, le score est de 0.76 en moyenne, avec un écart-type de 0.19.


# Tokenization des tweets

In [6]:
#Nombre de mots différents dans l'ensemble des articles 

##Récupération de tout les mots de tout les tweets
arr=df.Tweet.apply(lambda x : x.split(' ')).array
arr = reduce(add, arr)


##Nombre de mots différents dans l'ensemble des articles 
print('En utilisant le séparateur \' \', on obtient {} mots dans l\'ensemble des tweets'.format(len(set(arr))))


En utilisant le séparateur ' ', on obtient 19602 mots dans l'ensemble des tweets


In [7]:
# Tokenization en utilisant le Tokenizer Tweeter

arr_tokens = df.Tweet.apply(lambda x: TweetTokenizer().tokenize(x)).array
arr_tokens = reduce(add, arr_tokens)
print('En utilisant le Tokenizer TweetTokenizer, on obtient un vocabulaire de {} mots.'.format(len(set(arr_tokens))))

En utilisant le Tokenizer TweetTokenizer, on obtient un vocabulaire de 15804 mots.


In [8]:
counter=collections.Counter(arr_tokens)

#Mots les plus fréquents 
print('Sans compter la ponctuation, les mots qui apparaissent le plus sont par ordre décroissant sont :')
print(' ')
list_mots=['warming','change','the','Climate','to','global','climate']
for mot in list_mots:
    print(mot+' ('+str(counter[mot])+') ')                       




Sans compter la ponctuation, les mots qui apparaissent le plus sont par ordre décroissant sont :
 
warming (1944) 
change (1887) 
the (1775) 
Climate (1765) 
to (1725) 
global (1654) 
climate (1604) 


# Nettoyage des Tokens

In [9]:
#On prend toutes les phrases de touts les texts, et on les concatène dans une liste, en les traitant auparavant

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def remove_hashtags(tokens):
    tokens= map(lambda x : x.replace('#',''),tokens) #map : parcours tout les tokens
    return list(tokens)

def remove_url(tokens): #pb pour https
    tokens= filter(lambda x: "http" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

def remove_html(tokens):
    tokens= filter(lambda x: x[0]+x[-1]!='<>',tokens)
    return list(tokens)

def remove_www(tokens):
    tokens= filter(lambda x: "www" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

'''
def remove_x95(clean_corpus):
    for sentence_r in range(len(clean_corpus)):
        sentence=clean_corpus[sentence_r]
        for x in range(len(sentence)):
            if '\x95' in sentence[x]:
                y=sentence[x].split('\x95')
                new_x=''
                for part_x in y:
                    new_x=new_x+part_x
                sentence[x]=new_x
        clean_corpus[sentence_r]=sentence
    return(clean_corpus)
'''
    

def clean_text_first(corpus):
    tok=TweetTokenizer()
    tokens=[]
    for sample in corpus:
        token=tok.tokenize(sample) 
        token=remove_url(token)
        token=remove_html(token)
        token=remove_hashtags(token)
        token=remove_www(token)
        token=list(map(lambda x : x.lower(),token)) #.lower() : met les majuscules en minuscules
        tokens.append(token) #ajout du token à l'ensemble des phrases
    return tokens

def clean_text_second(corpus,threshold): #On rajoute l'association des mots qui vont ensembles
    
    #clean les textes
    tokens=clean_text_first(corpus)
    
    #associer les mots
    phrases=Phrases(tokens,threshold=threshold) #On fait apprendre le modèle d'association sur tout les mots
    phraser=Phraser(phrases) #Outil pour associer
    
    clean_tokens=[]
    for token in tokens: #On parcours les phrases et on associe les mots
        new_tokens=phraser[token]
        clean_tokens.append(new_tokens)
        
    #tokens = remove_x95(tokens)
    
    return(clean_tokens)


In [10]:
clean_text_first(df.Tweet)

[['global',
  'warming',
  'report',
  'urges',
  'governments',
  'to',
  'act',
  '|',
  'brussels',
  ',',
  'belgium',
  '(',
  'ap',
  ')',
  '-',
  'the',
  'world',
  'faces',
  'increased',
  'hunger',
  'and',
  '..'],
 ['fighting', 'poverty', 'and', 'global', 'warming', 'in', 'africa'],
 ['carbon',
  'offsets',
  ':',
  'how',
  'a',
  'vatican',
  'forest',
  'failed',
  'to',
  'reduce',
  'global',
  'warming'],
 ['carbon',
  'offsets',
  ':',
  'how',
  'a',
  'vatican',
  'forest',
  'failed',
  'to',
  'reduce',
  'global',
  'warming'],
 ['uruguay',
  ':',
  'tools',
  'needed',
  'for',
  'those',
  'most',
  'vulnerable',
  'to',
  'climate',
  'change'],
 ['rt',
  '@sejorg',
  ':',
  'rt',
  '@jaymiheimbuch',
  ':',
  'ocean',
  'saltiness',
  'shows',
  'global',
  'warming',
  'is',
  'intensifying',
  'our',
  'water',
  'cycle'],
 ['global',
  'warming',
  'evidence',
  'all',
  'around',
  'us',
  '|',
  'a',
  'message',
  'to',
  'global',
  'warming',
  'den

In [11]:
clean_text_second(df.Tweet,threshold=1000)

[['global',
  'warming',
  'report',
  'urges',
  'governments',
  'to',
  'act',
  '|',
  'brussels',
  ',',
  'belgium',
  '(',
  'ap',
  ')',
  '-',
  'the',
  'world',
  'faces',
  'increased',
  'hunger',
  'and',
  '..'],
 ['fighting', 'poverty', 'and', 'global', 'warming', 'in', 'africa'],
 ['carbon',
  'offsets',
  ':',
  'how',
  'a',
  'vatican_forest',
  'failed',
  'to',
  'reduce',
  'global',
  'warming'],
 ['carbon',
  'offsets',
  ':',
  'how',
  'a',
  'vatican_forest',
  'failed',
  'to',
  'reduce',
  'global',
  'warming'],
 ['uruguay',
  ':',
  'tools',
  'needed',
  'for',
  'those',
  'most',
  'vulnerable',
  'to',
  'climate',
  'change'],
 ['rt',
  '@sejorg',
  ':',
  'rt',
  '@jaymiheimbuch',
  ':',
  'ocean',
  'saltiness',
  'shows',
  'global',
  'warming',
  'is',
  'intensifying',
  'our',
  'water',
  'cycle'],
 ['global',
  'warming',
  'evidence',
  'all',
  'around',
  'us',
  '|',
  'a',
  'message',
  'to',
  'global',
  'warming',
  'deniers',
  '

# Représentation des Tweets

## Avec Word2Vec

On utilise tout d'abord l'algorithme Word2Vec pour représenter ces tweets. Chaque mot à une représentation vectorielle. Pour chaque tweet, on fait la moyenne des vecteurs (chaque mot) inclut dans ce tweet.

In [12]:
#Cleaning des tweets
clean_text=clean_text_second(df.Tweet,threshold=1000)

print("Entrainement du modèle Word2Vec ...")
model = Word2Vec(clean_text, size=100, window=5, min_count=3, workers=4) 

model.train(clean_text, total_examples=len(clean_text), epochs=10) #réseau de neuronne du Word2Vec
model_wv = model.wv

Entrainement du modèle Word2Vec ...


In [13]:
# Représentation des tweets en moyennant les mots

def tokens2vectors(tokenCorpus):
    ''' transforms our X into a list of list of vec (2D array) '''
    new_sample = list()
    for sample in tokenCorpus:
        tweetVecs = list()
        for token in sample:
            try : tweetVecs.append(model_wv.get_vector(token)  )
            except: tweetVecs.append( np.zeros(100) ) 
        new_sample.append(np.mean(tweetVecs, axis=0))
    
    return np.array(new_sample)


X= tokens2vectors(clean_text)

Y=[]
for x in list(X):
    try: Y.append(list(x))
    except : pass
    
df_representation_W2v= pd.DataFrame(Y).T

/Users/salimyoussfi/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/salimyoussfi/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
df_representation_W2v

,0,1,2,3,4,5,6,7,8,9,...,6076,6077,6078,6079,6080,6081,6082,6083,6084,6085
0,0.144320,0.302060,0.226678,0.226678,-0.101536,0.176827,0.137901,0.006574,-0.061614,0.110177,...,0.150176,-0.057882,0.227701,0.366575,0.398840,0.014003,0.094569,0.241553,0.057945,0.266063
1,0.055245,0.144319,-0.094910,-0.094910,0.022054,-0.023060,0.158206,0.126129,-0.095354,-0.043220,...,-0.007681,0.053739,-0.005749,0.308635,-0.021414,-0.015315,-0.040604,0.085293,0.030026,0.061804
2,-0.418573,-0.339130,-0.257455,-0.257455,-0.062358,-0.224878,-0.282267,-0.212181,-0.115948,-0.267541,...,-0.185805,-0.031610,-0.236183,-0.199809,-0.401310,-0.357141,-0.157551,-0.225020,-0.164491,-0.204677
3,-0.208832,-0.340209,-0.533814,-0.533814,-0.578745,-0.637622,-0.344252,-0.687139,-0.448655,-0.261926,...,-0.302199,-0.374023,-0.288231,-0.698842,-0.475782,-0.290555,-0.203475,-0.213134,-0.309738,-0.452041
4,0.104717,0.395406,0.330478,0.330478,0.199036,0.284143,0.288766,0.275764,0.186524,0.234129,...,0.141088,0.116070,0.307699,0.181036,0.283263,0.481458,0.107535,0.219362,0.214005,0.287488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.017744,-0.031631,0.088444,0.088444,-0.401738,0.128597,0.009790,-0.559565,-0.097403,0.051045,...,0.000656,-0.308707,-0.071919,-0.834028,-0.109917,0.114447,0.067983,-0.206981,-0.014929,-0.208286
96,-0.046360,-0.074729,-0.071973,-0.071973,-0.038267,0.024916,-0.140318,0.105402,-0.027876,0.020883,...,-0.006645,-0.013637,0.122627,0.157801,0.009400,-0.164195,-0.043268,0.026295,-0.074236,0.044655
97,0.141944,-0.136861,-0.425472,-0.425472,-0.332690,-0.522845,-0.070470,-0.288110,-0.429732,-0.232479,...,-0.200117,-0.263377,-0.225193,0.054935,-0.217277,-0.289429,-0.164869,-0.012259,-0.257879,-0.160770
98,0.096967,0.050671,-0.286894,-0.286894,-0.148778,-0.214236,0.013256,-0.240946,-0.032254,0.147463,...,-0.019792,-0.044727,-0.225981,-0.377677,-0.219015,-0.386009,0.061102,-0.181335,-0.132638,-0.236306


In [15]:
# Etude de cette représentation via la cosinus similarité

def closest_to(word,n_top_similar):
    print('Les {} mots les plus proches de {} sont : '.format(n_top_similar,word))
    print(' ')
    for word in [w[0] for w in model_wv.most_similar(word,topn=n_top_similar)]:
        print(word)
        
closest_to('change',10)

closest_to('climate',10)

Les 10 mots les plus proches de change sont : 
 
exciting
align
reform
bank
giles
roadblock
lindsey
wwfus
focus
support
Les 10 mots les plus proches de climate sont : 
 
immigration
betting
advisory
extend
move
impacts
graham
reform
pew
push


# Une approche naïve avec TF IDF


Pour chaque mot, on a un vecteur le représentant. Pour réprésenter un tweet, on fait la moyenne des vecteurs correspondant à chacun de ces mots.

In [16]:
# Nettoyage de la base
corpus=list(df.Tweet)
list_text=clean_text_second(df.Tweet,threshold=100)
list_text = reduce(add, list_text)


#Formatage pour le package TF-IDF
corpus=clean_text_second(df.Tweet,threshold=2000)
corpus_new=[]
for tweet in corpus: 
    tweet_sentence=''
    for word in tweet:
        tweet_sentence=tweet_sentence+' '+word
    corpus_new.append(tweet_sentence)

In [17]:
#Application de la méthode TF-IDF

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus_new)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()


#Représentation de chaque mot 
df_idf = pd.DataFrame(denselist, columns=feature_names)


In [18]:
def tokens2vectors_idf(tokenCorpus):
    ''' transforms our X into a list of list of vec (2D array) '''
    new_sample = list()
    for sample in tqdm(tokenCorpus):
        tweetVecs = []
        for token in sample:
            try : tweetVecs.append(list(df_idf[token]))
            except: tweetVecs.append( np.zeros(df_idf.shape[0]) ) 
                
        new_sample.append(np.mean(np.array(tweetVecs) ,axis=0))
    
    return np.array(new_sample)

In [23]:
#Cleaning des tweets
clean_text=clean_text_second(df.Tweet,threshold=1000)

X=tokens2vectors_idf(clean_text)

Y=[]
for x in list(X):
    try: Y.append(list(x))
    except : pass
    
df_representation_Tf_idf= pd.DataFrame(Y).T

100%|██████████| 6090/6090 [02:01<00:00, 50.31it/s]


In [24]:
df_representation_Tf_idf

,0,1,2,3,4,5,6,7,8,9,...,6076,6077,6078,6079,6080,6081,6082,6083,6084,6085
0,0.165637,0.038047,0.022159,0.022159,0.008921,0.009101,0.024289,0.000000,0.005773,0.010156,...,0.018470,0.012934,0.016852,0.008434,0.030468,0.014561,0.015556,0.018470,0.021821,0.006331
1,0.023965,0.336138,0.026205,0.026205,0.000000,0.018016,0.038833,0.000000,0.031083,0.021659,...,0.012533,0.000000,0.028103,0.018565,0.036032,0.028826,0.013727,0.012533,0.021089,0.022601
2,0.015454,0.029015,0.188371,0.188371,0.012444,0.012694,0.025862,0.000000,0.008052,0.014166,...,0.014782,0.000000,0.012592,0.007254,0.042499,0.020311,0.009672,0.014782,0.013600,0.008831
3,0.015454,0.029015,0.188371,0.188371,0.012444,0.012694,0.025862,0.000000,0.008052,0.014166,...,0.014782,0.000000,0.012592,0.007254,0.042499,0.020311,0.009672,0.014782,0.013600,0.008831
4,0.006319,0.000000,0.012638,0.012638,0.260397,0.000000,0.006620,0.019164,0.030524,0.005792,...,0.006044,0.020431,0.012121,0.006844,0.017377,0.000000,0.000000,0.006044,0.005561,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0.008295,0.026071,0.016591,0.016591,0.000000,0.011406,0.017381,0.000000,0.013806,0.017383,...,0.007935,0.000000,0.006759,0.006518,0.022812,0.318411,0.008690,0.007935,0.007300,0.007935
6086,0.011417,0.015993,0.010177,0.010177,0.000000,0.006997,0.010662,0.000000,0.000000,0.008051,...,0.022976,0.025694,0.012313,0.011873,0.013994,0.011195,0.122110,0.016973,0.018866,0.004867
6087,0.016438,0.017708,0.018863,0.018863,0.007594,0.007747,0.015783,0.000000,0.004914,0.008646,...,0.022425,0.022020,0.019828,0.021819,0.025937,0.012396,0.020583,0.187701,0.020631,0.013777
6088,0.020677,0.031726,0.018477,0.018477,0.007439,0.007589,0.020254,0.000000,0.004814,0.012142,...,0.021967,0.027867,0.020113,0.019833,0.025407,0.012142,0.024360,0.021967,0.185033,0.005279


# Avec BERT

# Avec du clustering